# MMG model

In [ ]:
%%time
# %load imports.py
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
import matplotlib

import sympy as sp
from src.symbols import *

#import src.models.MMG_rudder as rudder
from src.models.MMG_rudder import *
from src.models.MMG_propeller import *

#from src.models.vmm_MMG import *
from src.substitute_dynamic_symbols import run
from src.prime_system import PrimeSystem

In [ ]:
from src.models.MMG_hull import *

## Propeller

In [ ]:
eq_Xp

In [ ]:
eq_T

In [ ]:
eq_K_T

In [ ]:
eq_J

In [ ]:
eq_w_p

In [ ]:
X_P_solution[0][X_P]

In [ ]:
X_P_solution[0][K_T]

## Rudder

In [ ]:
eq_F_N

In [ ]:
eq_U_R

In [ ]:
eq_alpha_R

In [ ]:
eq_v_R

In [ ]:
eq_beta_R

In [ ]:
eq_u_R

In [ ]:
eq_v_R

In [ ]:
eq_alpha_R

In [ ]:
eq_U_R

In [ ]:
eq_F_N

In [ ]:
eq_f_alpha

## Implementation

In [ ]:
lambda_X_P = sp.lambdify(list(X_P_solution[0][X_P].free_symbols), X_P_solution[0][X_P])

lambda_X_R = sp.lambdify(list(X_R_solution.rhs.free_symbols), X_R_solution.rhs)
lambda_Y_R = sp.lambdify(list(Y_R_solution.rhs.free_symbols), Y_R_solution.rhs)
lambda_N_R = sp.lambdify(list(N_R_solution.rhs.free_symbols), N_R_solution.rhs)
K_T_solution = X_P_solution[0][K_T]
lambda_K_T = sp.lambdify(list(K_T_solution.free_symbols), K_T_solution)
w_p_solution = X_P_solution[0][w_p]
lambda_w_p = sp.lambdify(list(w_p_solution.free_symbols), w_p_solution)
lambda_beta_p = sp.lambdify(list(eq_beta_p.rhs.free_symbols), eq_beta_p.rhs)

lambda_thrust = sp.lambdify(list(X_P_solution[0][thrust].free_symbols), X_P_solution[0][thrust])

In [ ]:
lambda_K_T

In [ ]:
lambda_K_T

In [ ]:
lambda_X_R

In [ ]:
lambda_w_p

In [ ]:
%reload_kedro
derivatives = catalog.load("kvlcc2.derivatives")
ship_data = catalog.load("kvlcc2.ship_data")
propeller_coefficients = catalog.load("kvlcc2.propeller_coefficients_captive")
open_water_characteristics = catalog.load("kvlcc2.open_water_characteristics_captive")
ship_data['w_p0'] = 0
ship_data['tdf'] = 0

#propeller_coefficients = {key:10*value for key,value in propeller_coefficients.items()}

In [ ]:
ship_data

In [ ]:
derivatives

### Propeller

In [ ]:
lambda_X_P

In [ ]:
df_ = pd.DataFrame()
J_ = open_water_characteristics.index
n_ = 1.0
D_ = ship_data['D']
w0_ = ship_data['w_p0']
rho_ = 1000
tdf_ = ship_data['tdf']
df_['J'] = J_
df_['u'] = J_*n_*D_/(1-w0_)
df_['v'] = 0
df_['r'] = 0
df_['rev'] = n_
df_['U'] = np.sqrt(df_['u']**2 + df_['v']**2)

df_['beta'] = -np.arctan2(df_['v'], df_['u'])
ps = PrimeSystem(**ship_data)
xp_prime = ps._prime(
    ship_data["x_p"],
    unit="length",
)
r_prime = ps._prime(df_['r'], unit="angular_velocity", U=df_["U"])
df_['beta_p'] = df_['beta'] - xp_prime * r_prime


df_['C_2'] = np.where(df_['beta_p'] > 0, ship_data["C_2_beta_p_pos"], ship_data["C_2_beta_p_neg"])
df_.set_index('J', inplace=True)

In [ ]:
df_['fx'] = run(function=lambda_X_P, inputs=df_, **ship_data, **propeller_coefficients)

In [ ]:
propeller_coefficients

In [ ]:
df_['Kt'] = df_['fx'] / ((1 - ship_data['tdf']) * rho_ * n_ ** 2 * D_ ** 4)

fig,ax=plt.subplots()
open_water_characteristics.plot(y='Kt', label='propeller characteristics', ax=ax)
df_.plot(y='Kt', ax=ax)

### Rudder

In [ ]:
%reload_kedro
captive = catalog.load("kvlcc2.captive")

In [ ]:
captive.tail()

In [ ]:
df_rudder = captive.groupby(by='test type').get_group('rudder angle')
fig,ax=plt.subplots()
for V_, group in df_rudder.groupby(by='V'):
    group.sort_values(by='delta').plot(x='delta', y='fy', style='.-', ax=ax)

In [ ]:
%reload_kedro
ship_data = catalog.load("kvlcc2.ship_data")
ship_data.pop('C_2_beta_p_pos')
ship_data.pop('C_2_beta_p_neg')


In [ ]:
J_solution = X_P_solution[0][J]
lambda_J = sp.lambdify(list(J_solution.free_symbols), J_solution)
lambda_Lambda = sp.lambdify(list(eq_Lambda.rhs.free_symbols), eq_Lambda.rhs)
lambda_beta_R = sp.lambdify(list(eq_beta_R.rhs.free_symbols), eq_beta_R)
lambda_C_Th = sp.lambdify(list(eq_C_Th1.rhs.free_symbols),eq_C_Th1.rhs)

In [ ]:
#df_input = captive.drop(columns=['rev'])
#df_input = ps.prime(df_input, U=captive['V'])
df_input=captive.copy()
df_input['U'] = df_input['V']
xp_prime = ps._prime(
    ship_data["x_p"],
    unit="length",
)
r_prime = ps._prime(df_input['r'], unit="angular_velocity", U=df_input["V"])
df_input['beta_p'] = df_input['beta'] - xp_prime * r_prime


df_input['C_2'] = np.where(df_input['beta_p'] >= 0, derivatives["C_2_beta_p_pos"], derivatives["C_2_beta_p_neg"])
df_input['w_p'] = run(function=lambda_w_p, inputs=df_input, **ship_data, **propeller_coefficients)
df_input['J'] = run(function=lambda_J, inputs=df_input, **ship_data, **propeller_coefficients)
df_input['K_T'] = run(function=lambda_K_T, inputs=df_input, **ship_data, **propeller_coefficients)
df_input['gamma_R'] = np.where(df_input['beta_p'] >= 0, derivatives["gamma_R_B_R_pos"], derivatives["gamma_R_B_R_neg"])
df_input['Lambda'] = run(function=lambda_Lambda, inputs=ship_data)
r_prime = ps._prime(df_input['r'], unit='angular_velocity', U=df_input['U'])
df_input['beta_R'] = lambda_beta_R(l_R=derivatives['l_R'], r=r_prime, beta=df_input['beta'])
df_input['C_Th'] = run(function=lambda_C_Th, inputs=df_input, **ship_data)
df_input['thrust'] = run(function=lambda_thrust, inputs=df_input, **ship_data, **propeller_coefficients, **derivatives)

In [ ]:
df_input['X_R'] = run(function=lambda_X_R, inputs=df_input, **ship_data, **propeller_coefficients, **derivatives)
df_input['Y_R'] = run(function=lambda_Y_R, inputs=df_input, **ship_data, **propeller_coefficients, **derivatives)
df_input['N_R'] = run(function=lambda_N_R, inputs=df_input, **ship_data, **propeller_coefficients, **derivatives)
df_input['X_P'] = run(function=lambda_X_P, inputs=df_input, **ship_data, **propeller_coefficients)

In [ ]:
fig,ax=plt.subplots()
for V_, group in df_input.groupby(by='V'):
    group.sort_values(by='delta').plot(x='delta', y='Y_R', style='.-', ax=ax)

In [ ]:
fig,ax=plt.subplots()
for V_, group in df_rudder.groupby(by='V'):
    group.sort_values(by='delta').plot(x='delta', y='Y_R', style='.-', ax=ax)

## Hull

In [ ]:
eq_X_H

In [ ]:
subs = {value: key for key, value in p.items()}
eq = eq_X_H.rhs.subs(subs)
lambda_X_H = sp.lambdify(list(eq.free_symbols), eq)
eq = eq_Y_H.rhs.subs(subs)
lambda_Y_H = sp.lambdify(list(eq.free_symbols), eq)
eq = eq_N_H.rhs.subs(subs)
lambda_N_H = sp.lambdify(list(eq.free_symbols), eq)

In [ ]:
#df_input = captive.groupby(by='test type').get_group('drift angle').copy()
df_input_prime = ps.prime(df_input[['u','v','r']], U=df_input['V'])

In [ ]:
df_input_prime['X_H'] = run(function=lambda_X_H, inputs=df_input_prime, **derivatives)
df_input_prime['Y_H'] = run(function=lambda_Y_H, inputs=df_input_prime, **derivatives)
df_input_prime['N_H'] = run(function=lambda_N_H, inputs=df_input_prime, **derivatives)

In [ ]:
df_result = df_input.copy()
df_result['X_H'] = ps._unprime(df_input_prime['X_H'], unit='force', U=df_input['V'])
df_result['Y_H'] = ps._unprime(df_input_prime['Y_H'], unit='force', U=df_input['V'])
df_result['N_H'] = ps._unprime(df_input_prime['N_H'], unit='moment', U=df_input['V'])

In [ ]:
df_result['fx'] = df_result['X_H'] + df_result['X_R'] + df_result['X_P']
df_result['fy'] = df_result['Y_H'] + df_result['Y_R']
df_result['mz'] = df_result['N_H'] + df_result['N_R']

In [ ]:
df_result['test type'].unique()

In [ ]:
def plotter(group_, x, ax):
    
    plot = group_.sort_values(by=x).plot(x=x, y=dof, label=f"model V:{np.round(speed,1)} m/s", ax=ax)
    color = plot.axes.lines[-1].get_color()
    captive.loc[group_.index].sort_values(by=x).plot(x=x, y=dof, style=':o', color=color, label=f"PMM V:{np.round(speed,1)} m/s", ax=ax)
    ax.get_legend().set_visible(False)

xs = {'rudder angle':'delta',
     'drift angle':'beta',
      'drift + rudder angle':'delta + beta',
     }

df_result['delta + beta'] = df_result['delta'] + df_result['beta']
captive['delta + beta'] = captive['delta'] + captive['beta']

for test_type, group in df_result.groupby('test type'):
    
    fig,axes=plt.subplots(ncols=5)
    fig.set_size_inches(20,6)
    
    axes[0].plot([],'k-')
    axes[0].plot([],'k:o')
        
    for ax, dof in zip(axes, ['fx','fy','mz','Y_R','thrust']):
        
        for speed, group_ in group.groupby(by='V'):
            x = xs[test_type]
            if x=='beta':
                plotter(group_, x, ax)
            else:
                for _, group__ in group_.groupby(by='beta'):
                    plotter(group__, x, ax)
                        
            
            ax.set_title(dof)
    fig.suptitle(test_type)
            
    axes[0].legend(['model','PMM'])
    
        

In [ ]:
eq_w_p

In [ ]:
eq_u_R

## Regression

In [ ]:
from wPCC_pipeline.pipelines.force_regression.nodes import fit_forces

In [ ]:
%reload_kedro
vmm_name = 'vmm_martin'
vmm = catalog.load(vmm_name)
added_masses = catalog.load('kvlcc2.added_masses')
exclude_parameters = catalog.load("params:kvlcc2.motion_regression.exclude_parameters")

In [ ]:
#regression, parameters = fit_forces(data=captive, added_masses=added_masses, ship_data=ship_data, vmm=vmm, exclude_parameters = exclude_parameters)
regression, parameters = fit_forces(data=captive, added_masses=added_masses, ship_data=ship_data, vmm=vmm)

In [ ]:
regression.plot_pred_X()

In [ ]:
regression.plot_pred_Y()

In [ ]:
regression.plot_pred_N()

In [ ]:
regression.results_summary_Y()

In [ ]:
%reload_kedro
motion_regression = catalog.load(f"kvlcc2.{ vmm_name }.joined.model")

In [ ]:
df_parameters = pd.DataFrame()
df_parameters['motion regression'] = motion_regression.parameters
df_parameters['PMM'] = regression.parameters['regressed']
mask = (df_parameters.index.str.contains('r')) & ~(df_parameters.index.str.contains('thrust')) | (df_parameters.index.str.contains('dot'))
df_parameters = df_parameters.loc[~mask].copy()

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(15,6)
df_parameters.plot.bar(ax=ax)

In [ ]:
df_parameters.loc['Xthrust','PMM']